## Read data

In [ ]:
from csv_to_pandas import read_augmented_availability_data, read_station_data
df = read_augmented_availability_data("data.csv", "stations.csv")
stations = read_station_data("stations.csv")

## Plotting functions

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt

def plot_station_usage(df, stations):
    mpl.style.use("seaborn")
    for station_id in stations:
        sub_df = df[df.station_id == station_id]
        sub_df['real_capacity'] = sub_df.num_bikes_available + sub_df.num_docks_available
        sub_df[['num_bikes_available', 'real_capacity']].plot()
        top_row = sub_df.iloc[0]
        plt.ylim(0, top_row.station_capacity)
        plt.title(top_row.station_name)
        plt.show();

## Display_plots

In [ ]:
plot_station_usage(df, stations)